In [33]:
(defun row-sum-odd-numbers (n)
    (let* ((k (1- n))
           (row-start (+ (* k k) k 1)))
      (+ (* row-start n) (* k n))))

ROW-SUM-ODD-NUMBERS

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ROW-SUM-ODD-NUMBERS in DEFUN


In [35]:
(defun row-sum-odd-numbers (n)
  ;; 1 + 7*(n-1) + 6*(n-1)*(n-2) + (n-1)*(n-2)*(n-3)
  (+ 1 (* 7 (1- n)) (* 6 (1- n) (- n 2)) (* (1- n) (- n 2) (- n 3))))

ROW-SUM-ODD-NUMBERS

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ROW-SUM-ODD-NUMBERS in DEFUN


In [37]:
(loop :for n :from 1 to 10 :do (format t "~D " (row-sum-odd-numbers n)))

NIL

1 8 27 64 125 216 343 512 729 1000 

In [36]:
(loop :for n :from 1 to 20 :do (format t "~D " (row-sum-odd-numbers n)))

NIL

1 8 27 64 125 216 343 512 729 1000 1331 1728 2197 2744 3375 4096 4913 5832 6859 8000 

In [ ]:
(row-sum-odd-numbers 2)